# SCA demonstration

Focus of this notebook is to give a demonstration of the actual attack along with comparison of two different architectures used in the project. The only part of the attack that is missing is the collection of the traces from physical device.

Models are already trained in notebooks `1_0_SCA_MLP.ipynb` and `1_1_SCA_CNN.ipynb`. Full attacks were also implemented in those notebooks, but for easier demonstration, they will be repeated here as well.

## Load data


In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
import random
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import os
import numpy as np
import h5py
import tensorflow as tf
import keras
from datetime import datetime
import keras_tuner as kt
from tqdm.notebook import tqdm
import tensorflow_addons as tfa
import keras_tuner as kt

In [77]:
from utils.data_preparation import SCAML_Dataset
from utils.AES import AES
from utils.training import SCA_Trainer

In [84]:
EXECUTE_IN_COLAB = False
ATTACK_ALGORITHM = 'tinyaes'

if EXECUTE_IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    colab_root_path = '/content/drive/MyDrive/'
    
    training_data_path = colab_root_path + 'datasets/tinyaes/train'
    testing_data_path = colab_root_path + 'datasets/tinyaes/test'
    
    save_models_root_path = colab_root_path + 'models/'
    
    log_root_path = colab_root_path + 'logs/'
else:
    data_root_path = './data/SCA_datasets/datasets/' + ATTACK_ALGORITHM
    
    training_data_path = data_root_path + '/train'
    testing_data_path = data_root_path + '/test'
    
    save_models_root_path = './models/'
    
    log_root_path = './logs/'

In [131]:
dataset = SCAML_Dataset()

dataset.create_dataset(training_data_path, attack_point='sub_bytes_out', trace_length=8000)
dataset.create_dataset(testing_data_path, attack_point='sub_bytes_out', attack=True, trace_length=8000)

256it [00:03, 74.22it/s]
Loading shards: 100%|████████████████████████████████████████████████████████████████| 256/256 [01:16<00:00,  3.34it/s]
256it [00:02, 110.11it/s]
Loading shards: 100%|████████████████████████████████████████████████████████████████| 256/256 [01:23<00:00,  3.06it/s]


# Conclusions

Applying machine learning techniques to cryptanalysis is relatively new concept, and there is a lot of research happening in this area. It is obvious that this is still a hard and only partially explored topic. Making Side Channell Attacks easier could help hardware (and software) developers to develop products that are more resillient and secure. Of course, as is often case in security, there is a constant race between attackers and defenders and tools such as techniques listed in this project could be useful for both sides.

Some of the takeaways from this project:
* Using Keras hypertuner proved to be beneficial when finding the right model architecture. 
* Convolutional models performed better than simple MLPs.
* Convolutional models were able to learn internal features of the data without any preprocessing
* Other architectures could further improve performance of the attacks, but were out of scope at the time of writing this
* ML based techniques are simpler to setup and understand, but very hard to train properly

# Further research:

Deep-Cryptanalysis - Fashion or Revolution? [video](https://www.youtube.com/watch?v=hUmsyaPU-qU&t=2s)

Unsupervised Learning SCA: [video](https://www.youtube.com/watch?v=so-18y5Avr0)/[paper](https://arxiv.org/pdf/2001.05951.pdf)

Tips on tuning neural networks SCA [video](https://www.youtube.com/watch?v=uSpFfacjU4g)